# Project Part 2

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/brearenee/NLP-Project/blob/main/part2-startrek.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/https://github.com/brearenee/NLP-Project/blob/main/part2-startrek.ipynb)


**NLP Problem:** given a script from Star Trek The Next Generation, predict from 8 main characters who said a line. 

Part 2 of my project involves creating a basic model for my NLP problem


As mentioned in Part 1, my dataset's original format is not in the most useful form.  
To start Part 2, I must parse through the raw JSON and return an organized dataFrame. 

In [1]:
import pandas as pd
import json
import requests
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
url = 'https://raw.githubusercontent.com/brearenee/NLP-Project/main/dataset/StarTrekDialogue_v2.json'
response = requests.get(url)

##This CodeBlock is thanks to ChatGPT :-) 
if response.status_code == 200:
    json_data = json.loads(response.text)
    lines = []
    characters = []
    episodes = []
  
    # extract the information from the JSON file for the "TNG" series
    for series_name, series_data in json_data.items():
        if series_name == "TNG": 
            for episode_name, episode_data in series_data.items():
                for character_name, character_lines in episode_data.items():
                    for line_text in character_lines:
                        lines.append(line_text)
                        characters.append(character_name)
                        episodes.append(episode_name)
                     
    # Create a DataFrame from the extracted data
    df = pd.DataFrame({
        'Line': lines,
        'Character': characters,
        'Episode': episodes,
    })

    # Remove duplicate lines, keeping the first occurrence (preserving the original order)
    df = df.drop_duplicates(subset='Line', keep='first')

    # Reset the index of the DataFrame
    df.reset_index(drop=True, inplace=True)

else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


We then need to clean our dataset by removing non-main characters.  We are going to remove all characters that have less than 1000 lines. 


In [3]:
character_counts = df['Character'].value_counts()

characters_to_remove = character_counts[character_counts < 1000].index
df = df[~df['Character'].isin(characters_to_remove)]


In [4]:
df['Character'].value_counts()


Character
PICARD     10798
RIKER       6454
DATA        5699
LAFORGE     4111
WORF        3185
CRUSHER     2944
TROI        2856
WESLEY      1206
Name: count, dtype: int64

## Tokenizing 
Before creating our model, we need to initiate the tokenization process for the "Lines" field in our dataframe. This involves breaking down each sentence into individual tokens, enhancing the model's ability to interpret and analyze them.

For the current phase, we're retaining stop words. There’s a chance they might hold stylistic nuances that are important for character prediction.  However, recognizing that our dataset is sourced from the internet, we'll enforce consistency by converting all text in the "Line" field to lowercase.


In [5]:
nltk.download('punkt')
df['Line'] = df['Line'].apply(lambda x: word_tokenize(x.lower()))


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
